**GDAL** (Geospatial Data Abstraction Library) is a translator library for raster and vector geospatial data formats that is released under an MIT style Open Source License by the Open Source Geospatial Foundation. As a library, it presents a single raster abstract data model and single vector abstract data model to the calling application for all supported formats. It also comes with a variety of useful command line utilities for data translation and processing.

Official website: https://gdal.org/index.html

There is the package for Python: https://pypi.org/project/GDAL/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install osgeo

In [ ]:
%matplotlib inline
from osgeo import gdal_array
import gdal
from osgeo.gdalnumeric import *
import numpy as np
import matplotlib.pyplot as plt

## common imports

In [ ]:
np.random.seed(100)

directory = '/content/drive/MyDrive/GIS_tutorials/2_python_main_gis_functions/data_package_beg/'
fileName = directory + "out1.tif"


In [ ]:
datain = fileName

raster_ds = gdal.Open(datain, gdal.GA_ReadOnly)
image_gdal = raster_ds.GetRasterBand(1).ReadAsArray() ## get the first band
print(image_gdal.shape)

plt.imshow(image_gdal)
plt.colorbar()
plt.show()

In [ ]:
## if we use the shape from above we can demonstrate that satellite images are just really numpy arrays!

random_image = np.random.random([2272, 3847])
plt.imshow(random_image, interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
## did that prove anything?

newimage = (image_gdal * random_image**2) 
plt.imshow(newimage)
plt.show()

## does this? Only that yes you can alter images using numpy and fast!

## bonus visit https://speakerdeck.com/jakevdp/seven-strategies-for-optimizing-numerical-code
## one of the take homes is that if you can convert to numpy arrays!

Raster statistics

In [ ]:
## getting statistics

# rows and columns
ncol = raster_ds.RasterXSize
nrow = raster_ds.RasterYSize

# projection and extent
proj = raster_ds.GetProjectionRef()
ext = raster_ds.GetGeoTransform()

ulx, xres, xskew, uly, yskew, yres  = raster_ds.GetGeoTransform()
lrx = ulx + (raster_ds.RasterXSize * xres)
lry = uly + (raster_ds.RasterYSize * yres)
print(ulx, uly, lrx, lry)
# raster_ds = None

## what has this actually told you?

In [ ]:
img_ds = gdal.Open(datain, gdal.GA_ReadOnly)

img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))


print(gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))

print(img.shape)

## the hardest bit??
for b in range(img.shape[2]):
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()


plt.imshow(img)
plt.show()

99% clip - we will revisit this later

In [ ]:
minv = np.percentile(img, 1)
maxv = np.percentile(img, 99)

print(minv)
print(maxv)

img2 = ((img - minv)/(maxv - minv))*255

print('max', np.nanmax(img2))
print('min', np.nanmin(img2))
print('mean', np.nanmean(img2))

plt.imshow(img2.astype(np.uint8))

plt.show()

or this? on a single band

In [ ]:
print(img.shape)

band1 = img_ds.GetRasterBand(1).ReadAsArray()
band2 = img_ds.GetRasterBand(2).ReadAsArray()
band3 = img_ds.GetRasterBand(3).ReadAsArray()

minv1 = np.percentile(band1, 1)
maxv1 = np.percentile(band1, 99)

print(minv1)
print(maxv1)

band1_2 = ((band1 - minv1)/(maxv1 - minv1))*255

minv2 = np.percentile(band2, 1)
maxv2 = np.percentile(band2, 99)
band2_2 = ((band2 - minv2)/(maxv2 - minv2))*255

minv3 = np.percentile(band3, 1)
maxv3 = np.percentile(band3, 99)
band3_2 = ((band3 - minv3)/(maxv3 - minv3))*255

#### show the data!!!
plt.figure(figsize=(10, 6))

plt.subplot(3, 2, 1)
plt.imshow(band1)

plt.subplot(3, 2, 2)
plt.imshow(band1_2)

plt.subplot(3, 2, 3)
plt.imshow(band2)

plt.subplot(3, 2, 4)
plt.imshow(band2_2)

plt.subplot(3, 2, 5)
plt.imshow(band3)

plt.subplot(3, 2, 6)
plt.imshow(band3_2)


plt.show()

# Writing out the data

In [ ]:
## write out to tiff
### single band raster. 

ds = gdal.Open(fileName)
band = ds.GetRasterBand(1)
arr = band.ReadAsArray()
[cols, rows] = arr.shape

format = "GTiff"
driver = gdal.GetDriverByName(format)


outDataRaster = driver.Create(directory + "out_b1_stretch1.tif", rows, cols, 1, gdal.GDT_Byte)

print(ds.GetGeoTransform())
print(type(ds.GetProjection()))
outDataRaster.SetGeoTransform(ds.GetGeoTransform())##sets same geotransform as input
outDataRaster.SetProjection(ds.GetProjection())##sets same projection as input


outDataRaster.GetRasterBand(1).WriteArray(band1_2)

outDataRaster.FlushCache() ## remove from memory
del outDataRaster ## delete the data (not the actual geotiff)